In [121]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from typing import Iterable, Callable
from questionanswering import squad

In [122]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [123]:
df = squad.parse_json_file("input/train-v2.0.json")
questions = list(df["question"])
contexts = list(df["context"])
df.info()

100%|████████████████████████████████| 442/442 [00:00<00:00, 732.84it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            130319 non-null  object
 1   title         130319 non-null  object
 2   question      130319 non-null  object
 3   answer_text   130319 non-null  object
 4   context       130319 non-null  object
 5   answer_start  130319 non-null  int16 
dtypes: int16(1), object(5)
memory usage: 5.2+ MB


In [124]:
assert len(df) == 130319
assert len(df) == len(set(df["id"]))
df.set_index("id", drop=False, inplace=True)
df.head()

,id,title,question,answer_text,context,answer_start
id,,,,,,
56be85543aeaaa14008c9063,56be85543aeaaa14008c9063,Beyoncé,When did Beyonce start becoming popular?,in the late 1990s,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",269
56be85543aeaaa14008c9065,56be85543aeaaa14008c9065,Beyoncé,What areas did Beyonce compete in when she was growing up?,singing and dancing,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",207
56be85543aeaaa14008c9066,56be85543aeaaa14008c9066,Beyoncé,When did Beyonce leave Destiny's Child and become a solo singer?,2003,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",526
56bf6b0f3aeaaa14008c9601,56bf6b0f3aeaaa14008c9601,Beyoncé,In what city and state did Beyonce grow up?,"Houston, Texas","Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",166
56bf6b0f3aeaaa14008c9602,56bf6b0f3aeaaa14008c9602,Beyoncé,In which decade did Beyonce become famous?,late 1990s,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, 

In [125]:
s = pd.Series(np.where(df["answer_start"]==-1, True, False))
assert s.value_counts()[True] == 43498
s.value_counts(normalize=True)

False    0.666219
True     0.333781
dtype: float64

# Answer corrections

In [126]:
%%time
df.loc["56bf7e603aeaaa14008c9681", "answer_text"] = "split with Luckett and Roberson"
df.loc["56be973d3aeaaa14008c9123", "answer_text"] = "sixth"
df.loc["56d3ac8e2ccc5a1400d82e1b", "answer_text"] = "operatic"
df.loc["56cbdea66d243a140015edae", "answer_text"] = "seven"
df.loc["56cbdea66d243a140015edae", "answer_start"] = 332
df.loc["56cf609aaab44d1400b89187", "answer_text"] = "seven"
df.loc["56cf609aaab44d1400b89187", "answer_start"] = 332
df.loc["56d20a6ae7d4791d0090261a", "answer_text"] = "seven"
df.loc["56d20a6ae7d4791d0090261a", "answer_start"] = 332
df.loc["56ccde7862d2951400fa64d9", "answer_start"] = 790
df.loc["56cd682162d2951400fa658e", "answer_start"] = 463
df.loc["56ce726faab44d1400b88793", "answer_text"] = "three"
df.loc["56ce726faab44d1400b88793", "answer_start"] = 302
df.loc["56cc57466d243a140015ef24", "question"] = "in which year did the sales of iPhone exceed iPod?"
df.loc["56cc57466d243a140015ef24", "answer_text"] = "2010"
df.loc["56cc57466d243a140015ef24", "answer_start"] = 444
df.loc["56ce750daab44d1400b887b4", "answer_start"] = 396
df.loc["56cd5d3a62d2951400fa653e", "answer_text"] = "manually"
df.loc["56cd8ffa62d2951400fa6723", "question"] = "what is Akiko Kōmoto's nationality?"
df.loc["56cd8ffa62d2951400fa6723", "answer_text"] = "Japanese"
df.loc["56cf2e15aab44d1400b88dc9", "answer_text"] = "twenty-fourth"
df.loc["56cf2e15aab44d1400b88dcb", "answer_text"] = "fourth"
df.loc["56cf2e15aab44d1400b88dcb", "answer_start"] = 113
df.loc["56cdcf7d62d2951400fa686d", "answer_start"] = 523
df.loc["56cdcf7d62d2951400fa686e", "answer_start"] = 571
df.loc["56cf4113aab44d1400b88ee0", "answer_text"] = "fourth"
df.loc["56cf469eaab44d1400b88f00", "answer_text"] = "sixth"
df.loc["56cde1f462d2951400fa6961", "answer_text"] = "two"
df.loc["56cde1f462d2951400fa6961", "answer_start"] = 537
df.loc["56cebbdeaab44d1400b8895c", "answer_text"] = "a million"
df.loc["56cebbdeaab44d1400b8895c", "answer_start"] = 268
df.loc["56cf1a05aab44d1400b88d7f", "answer_start"] = 332
df.loc["56cf3442aab44d1400b88e39", "answer_start"] = 352
df.loc["56cfe987234ae51400d9c09b", "answer_text"] = "92%"
df.loc["56cfe987234ae51400d9c09b", "answer_start"] = 444
df.loc["56cf36e5aab44d1400b88e71", "answer_start"] = 618
df.loc["56cffb63234ae51400d9c1e7", "answer_text"] = "Two"
df.loc["56cffb63234ae51400d9c1e7", "answer_start"] = 137
df.loc["56cf572daab44d1400b89072", "answer_text"] = "41st"
df.loc["56cf5c2caab44d1400b8911d", "answer_start"] = 164
df.loc["56d3f37f2ccc5a1400d82f85", "answer_start"] = 164
df.loc["56cf4e9aaab44d1400b88f9c", "answer_text"] = "seventh"
df.loc["56cf4e9aaab44d1400b88f9c", "answer_start"] = 983
df.loc["56d0f47a17492d1400aab69d", "answer_text"] = "seventh"
df.loc["56d0f47a17492d1400aab69d", "answer_start"] = 983
df.loc["56cf58f7aab44d1400b890b8", "answer_text"] = "Three"
df.loc["56cf58f7aab44d1400b890b8", "answer_start"] = 280
df.loc["56d0f8a117492d1400aab6af", "answer_text"] = "Three"
df.loc["56d0f8a117492d1400aab6af", "answer_start"] = 280
df.loc["56cf64d24df3c31400b0d6f5", "answer_text"] = "three"
df.loc["56cf64d24df3c31400b0d6f5", "answer_start"] = 925
df.loc["56d1042317492d1400aab72f", "answer_text"] = "three"
df.loc["56d1042317492d1400aab72f", "answer_start"] = 925
df.loc["56cf657b4df3c31400b0d6ff", "answer_text"] = "two"
df.loc["56cf657b4df3c31400b0d6ff", "answer_start"] = 94
df.loc["56d1056017492d1400aab755", "answer_start"] = 1071
df.loc["56cf79be4df3c31400b0d80e", "answer_text"] = "two"
df.loc["56cf79be4df3c31400b0d80e", "answer_start"] = 370
df.loc["56d11a9217492d1400aab976", "answer_text"] = "two"
df.loc["56d11a9217492d1400aab976", "answer_start"] = 370
df.loc["56cf829c234ae51400d9bdd4", "answer_text"] = "two"
df.loc["56cf829c234ae51400d9bdd4", "answer_start"] = 589
df.loc["56d131ad17492d1400aabbd3", "answer_text"] = "two"
df.loc["56d131ad17492d1400aabbd3", "answer_start"] = 943
df.loc["56cf8915234ae51400d9be15", "answer_text"] = "three"
df.loc["56cf8915234ae51400d9be15", "answer_start"] = 77
df.loc["56d13983e7d4791d00902041", "answer_text"] = "three"
df.loc["56d13983e7d4791d00902041", "answer_start"] = 77
df.loc["56d1b926e7d4791d009020cf", "answer_text"] = "Four"
df.loc["56d1b926e7d4791d009020cf", "answer_start"] = 787
df.loc["56d1c2d2e7d4791d00902121", "answer_text"] = "5th century CE"
df.loc["56d24a6fb329da140004ed00", "answer_start"] = 513
df.loc["56d2641d59d6e41400145f54", "answer_start"] = 453
df.loc["56d366ee59d6e414001462f6", "answer_text"] = "fifteenth"
df.loc["56d366ee59d6e414001462f6", "answer_start"] = 333
df.loc["56d36e4b59d6e4140014638c", "answer_text"] = "fourteenth"
df.loc["56db2190e7c41114004b4dd5", "answer_text"] = "fourth"
df.loc["56d383b159d6e414001465e6", "answer_text"] = "fifteenth"
df.loc["56d383b159d6e414001465e6", "answer_start"] = 39
df.loc["56d383b159d6e414001465e7", "answer_text"] = "fifth"
df.loc["56d383b159d6e414001465e7", "answer_start"] = 326
df.loc["56d3881459d6e41400146671", "answer_text"] = "fifteenth"
df.loc["56d3881459d6e41400146671", "answer_start"] = 411
df.loc["56d3883859d6e41400146678", "question"] = "For how long was American Idol the highest rated reality show on television?"
df.loc["56d3883859d6e41400146678", "answer_text"] = "almost a decade"
df.loc["56d3883859d6e41400146678", "answer_start"] = 388
df.loc["56d3887c59d6e4140014667b", "answer_text"] = "eight"
df.loc["56d3887c59d6e4140014667b", "answer_start"] = 148
df.loc["56d38b4e59d6e414001466d7", "answer_text"] = "nine"
df.loc["56d38b4e59d6e414001466d7", "answer_start"] = 87
df.loc["56d38b4e59d6e414001466d9", "question"] = "In which year did American Idol win an award for Best Reality Competition?"
df.loc["56d38b4e59d6e414001466d9", "answer_text"] = "2011"
df.loc["56d38b4e59d6e414001466d9", "answer_start"] = 644
df.loc["56d5fc2a1c85041400946ea0", "answer_text"] = "Dog breeds"
df.loc["56d5fc2a1c85041400946ea0", "answer_start"] = 279
df.loc["56d62e521c85041400946f9f", "answer_text"] = "emotional"
df.loc["56d62f3e1c85041400946fa5", "answer_text"] = "hunting"
df.loc["56dafad1e7c41114004b4bfc", "answer_start"] = 310
df.loc["56dd1cf99a695914005b94d8", "answer_text"] = "southwestern"
df.loc["56dd1dc29a695914005b94e3", "answer_start"] = 621
df.loc["56dd1ef39a695914005b94f4", "answer_start"] = 349
df.loc["56de57394396321400ee2830", "answer_text"] = "sixth"
df.loc["56deebdc3277331400b4d81f", "question"] = "Each Brigade contains how many logistics battalions?"
df.loc["56deebdc3277331400b4d81f", "answer_start"] = 454
df.loc["56de0abc4396321400ee2563", "answer_text"] = "Islamic"
df.loc["56df844f56340a1900b29cca", "answer_text"] = "700 lumens"
df.loc["56df844f56340a1900b29cca", "answer_start"] = 909
df.loc["56df865956340a1900b29ceb", "question"] = "What technique considers the amount of daylight received indoors?"
df.loc["56df865956340a1900b29ceb", "answer_text"] = "Daylight factor calculation"
df.loc["56df865956340a1900b29ceb", "answer_start"] = 376
df.loc["56df95d44a1a83140091eb81", "question"] = "What is the effect of high GAI value?"
df.loc["56df95d44a1a83140091eb81", "answer_text"] = "the greater the apparent saturation or vividness of the object colors"
df.loc["56df95d44a1a83140091eb81", "answer_start"] = 310
df.loc["56dfa6de7aa994140058df9a", "question"] = "What are the medical outcomes for patients with natural scenery?"
df.loc["56dfa6de7aa994140058df9a", "answer_text"] = "shorter postoperative hospital stays, received fewer negative evaluative comments in nurses' notes, and took fewer potent analegesics"
df.loc["56dfa6de7aa994140058df9a", "answer_start"] = 245
df.loc["56dfa6de7aa994140058df9b", "question"] = "What is the key finding of the study?"
df.loc["56dfa6de7aa994140058df9b", "answer_text"] = "the nature of the scenery and daylight exposure was indeed healthier for patients"
df.loc["56dfa6de7aa994140058df9b", "answer_start"] = 487
df.loc["56dfbc1d231d4119001abd48", "answer_start"] = 420
df.loc["56e462c98c00841900fbaf71", "answer_start"] = 100
df.loc["56e4744d39bdeb140034793b", "answer_start"] = 102
df.loc["56e4793839bdeb140034794f", "question"] = "How should a building fulfil the contemporary ethos?"
df.loc["56e4793839bdeb140034794f", "answer_text"] = "constructed in a manner which is environmentally friendly"
df.loc["56e4793839bdeb140034794f", "answer_start"] = 238
df.loc["56df623596943c1400a5d494", "answer_start"] = 915







Wall time: 395 ms


In [127]:
%%time
col = "qc_length"
df[col] = df["question"].str.len() + df["context"].str.len() 
df[col] = df[col].astype(np.int32)

Wall time: 86.3 ms


In [128]:
%%time
col = "a_length"
df[col] = df["answer_text"].str.len()
df[col] = df[col].astype(np.int32)

Wall time: 54 ms


In [129]:
def word_length(cols: Iterable) -> Callable:
    def f(row) -> int:
        res = 0
        for col in cols:
            res += len(row[col].split())
        return res
    
    return f

In [130]:
%%time
col = "qc_word_length"
df[col] = df.progress_apply(word_length(["question", "context"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████| 130319/130319 [00:02<00:00, 51860.01it/s]

Wall time: 2.52 s


In [131]:
%%time
col = "a_word_length"
df[col] = df.progress_apply(word_length(["answer_text"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|███████████████████████| 130319/130319 [00:01<00:00, 111291.75it/s]

Wall time: 1.17 s


In [132]:
#pretrained_dir = "../pretrained/google/electra-small-discriminator"
#tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

In [133]:
#pretrained_dir = "../pretrained/albert-base-v2"
#sp_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(sp_tokenizer)}\n{sp_tokenizer.model_input_names}")

In [134]:
#pretrained_dir = "../pretrained/distilroberta-base"
#bpe_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(bpe_tokenizer)}\n{bpe_tokenizer.model_input_names}")

In [135]:
#%%time
#x = tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_wp_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [136]:
#%%time
#x = sp_tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_sp_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [137]:
#%%time
#x = bpe_tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_bpe_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [138]:
#cols = ["qc_length", "a_length", "qc_word_length", "a_word_length", 
#        "qc_wp_length", "qc_sp_length", "qc_bpe_length"]
#df[cols].describe(percentiles=percentiles)

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130319 entries, 56be85543aeaaa14008c9063 to 5a7e070b70df9f001a87543d
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              130319 non-null  object
 1   title           130319 non-null  object
 2   question        130319 non-null  object
 3   answer_text     130319 non-null  object
 4   context         130319 non-null  object
 5   answer_start    130319 non-null  int16 
 6   qc_length       130319 non-null  int32 
 7   a_length        130319 non-null  int32 
 8   qc_word_length  130319 non-null  int32 
 9   a_word_length   130319 non-null  int32 
dtypes: int16(1), int32(4), object(5)
memory usage: 12.2+ MB


In [140]:
%%time
df.to_parquet("output/train.parquet", index=False)

Wall time: 304 ms
